In [18]:
import time
import sqlite3
import requests
import datetime
import pandas as pd
import shioaji as sj
from shioaji import TickFOPv1, Exchange
from exchange_calendars import get_calendar
import mplfinance as mpf
import talib

from FuncBase import *

In [19]:
import os 
import shioaji as sj
from dotenv import load_dotenv

#永豐金API登入
load_dotenv()
api = sj.Shioaji(simulation=True)
api.login(os.getenv('YOUR_PERSON_ID'),os.getenv('YOUR_PASSWORD'),
    contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

#連結DB Browser
connection = sqlite3.connect('work_data.db')

[22:59:16.273000] [12552] [pysolace/core/sol.cpp:499:SendRequest] [E] [thread 142929] Not ready


Exception ignored in: <function Shioaji.__del__ at 0x7f7ac0e68b80>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/shioaji/shioaji.py", line 1211, in __del__
    self._solace.__del__()
  File "shioaji/backend/solace/api.pyx", line 2455, in shioaji.backend.solace.api.SolaceAPI.__del__
  File "shioaji/backend/solace/api.pyx", line 484, in shioaji.backend.solace.api.SolaceAPI.logout
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pysolace/__init__.py", line 275, in request
    raise TimeoutError(
TimeoutError: Topic: api/v1/auth/logout, Corr: c23, Client: PYAPI/F229985348/1014/061040/407937/124.218.210.253, payload: {'token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJuYmYiOjE3Mjg4ODYyNDAsImV4cCI6MTcyODk3MjY0MCwic2ltdWxhdGlvbiI6dHJ1ZSwicGVyc29uX2lkIjoiRjIyOTk4NTM0OCIsInZlcnNpb24iOiIwLjUuMC5kZXYyIiwicDJwIjoiI1AyUC92OmlkY3NvbGFjZTAxL09yeWZrcXFtL1BZQVBJL0YyMjk5ODUzNDgvMTAxNC8wNjEwNDAv

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done
<SecurityType.Stock: 'STK'> fetch done
<SecurityType.Future: 'FUT'> fetch done
<SecurityType.Option: 'OPT'> fetch done


In [7]:
open_orders = {}
order_qty = {}
positions = {}

<SecurityType.Stock: 'STK'> fetch done
<SecurityType.Option: 'OPT'> fetch done


In [8]:
def place_cb(stat, msg):

    print('stat', stat)
    print('msg', msg)
    #'TFTDEAL' 下單成功
    if stat.value == 'TFTDEAL':

        code = msg['code']
        qty = int(msg['quantity'])
                
        order_qty[code] -= qty #扣除下單時委託張數

        if order_qty[code] == 0:
            open_orders = None

        position = positions.get(code,0)

        if msg['action'] == 'Sell':
            qty = qty * -1

        position += qty

        positions[code] = position
    
api.set_order_callback(place_cb)


Buy order

In [11]:
code = 'TXF202410'
codes = 'TXF'
current_price = 22782
qty = 1

contract = api.Contracts.Futures.get(codes)[code]

order = api.Order(
                  action=sj.constant.Action.Buy, 
                  price=current_price,
                  quantity=qty,
                  price_type="LMT",
                  order_type="ROD",
                  octype=sj.constant.FuturesOCType.Auto,
                  account=api.futopt_account)

trade = api.place_order(contract, order)

open_orders[code] = trade
order_qty[code] = qty

print(open_orders)
print(order_qty)
print(positions)

statResponse Code: 0 | Event Code: 0 | Info: host '218.32.76.102:80', hostname '218.32.76.102:80' IP 218.32.76.102:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
 OrderState.FOrder
msg {'operation': {'op_type': 'New', 'op_code': '00', 'op_msg': ''}, 'order': {'id': '000003', 'seqno': '000003', 'ordno': '00002A', 'account': {'account_type': 'F', 'person_id': '', 'broker_id': 'F002000', 'account_id': '1553688', 'signed': False}, 'action': 'Buy', 'price': 22782, 'quantity': 1, 'order_type': 'ROD', 'price_type': 'LMT', 'oc_type': 'Auto', 'custom_field': ''}, 'status': {'id': '000003', 'exchange_ts': 1728886360.750907, 'order_quantity': 1, 'modified_price': 0, 'cancel_quantity': 0, 'web_id': 'Z'}, 'contract': {'security_type': 'FUT', 'exchange': 'TAIFEX', 'code': 'TXFJ4'}}
{'TXF202410': Trade(contract=Future(code='TXFJ4', symbol='TXF202410', name='臺股期貨10', category='TXF', delivery_month='202410', delivery_date='2024/10/16', underlying

Sell order

In [14]:
code = 'TXF202410'
codes = 'TXF'
current_price = 22782
position = positions.get(code, 0)

contract = api.Contracts.Futures.get(codes)[code]

order = api.Order(
                  action=sj.constant.Action.Sell, 
                  price=current_price,
                  quantity=positions,
                  price_type="LMT",
                  order_type="ROD",
                  octype=sj.constant.FuturesOCType.Auto,
                  account=api.futopt_account)

trade = api.place_order(contract, order)

open_orders[code] = trade
order_qty[code] = positions

print(open_orders)
print(order_qty)
print(positions)

{}

整合

In [20]:
ticks = {}
kbars = {}
volume_today = {}

open_orders = {}
order_qty = {}
positions = {}

#設定callback 回傳資料型態
@api.on_tick_fop_v1()
def quote_callback(exchange: Exchange, tick: TickFOPv1):

    ticks = {}
    volume_today = {}

    ts = pd.to_datetime(tick.datetime)

    if tick.code[0].isalpha():
        code = tick.code[0:3]
    else:
        code = tick.code

    ts = pd.to_datetime(tick.datetime)

    if 5 < ts.hour < 9:
        return
    
    close = float(tick.close)
    volume = tick.volume
    vol_sum = tick.total_volume

    
    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum

def place_cb(stat, msg):

    if stat.value == 'TFTDEAL':

        code = msg['code']
        qty = int(msg['quantity'])
                
        order_qty[code] -= qty #扣除下單時委託張數

        if order_qty[code] == 0:
            open_orders = None #委託張數歸0

        position = positions.get(code,0)

        if msg['action'] == 'Sell':
            qty = qty * -1

        position += qty

        positions[code] = position
    
api.set_order_callback(place_cb)


In [21]:
import pickle

entry_time = None
entry_price = 0

exit_time = None
exit_price = 0

AC = 0
EH = 0
last_price = 0
cost_after_add = 0

with open('trade_historical_variable.pkl', 'wb') as file:
    trade_his_var = {
                    'Entry_time': entry_time,
                    'Entry_P':entry_price,
                    'last_P': last_price,
                    'AC': AC,
                    'EH': EH,
                    'cost_after_add':cost_after_add,
                    'Position': positions.get(code, 0)}

    pickle.dump(trade_his_var, file)

print("資料已保存")

資料已保存


In [23]:
import time
import pickle

def tbot(codes, is_Futures = False,
         filter = 0.1, Ratio_Drawdown1 = 0.1, Ratio_Drawdown2 = -0.03, Quit_position = 0):

    print('--- 啟動 Tbot for {} ---'.format(codes))

    tw_calendar = get_calendar('XTAI')

    #Risk Variables
    filter = filter
    Ratio_Drawdown1 = Ratio_Drawdown1 #預設台股最大漲跌幅
    Ratio_Drawdown2 = Ratio_Drawdown2
    Quit_position = Quit_position

    #Call Other variable
    with open('trade_historical_variable.pkl', 'rb') as file:
        data = pickle.load(file)

    entry_time = data['Entry_time']
    entry_price = data['Entry_P']
    last_price = data['last_P']
    AC = data['AC']
    EH = data['EH']
    cost_after_add = data['cost_after_add']
    
    
    #Start
    date = pd.to_datetime(datetime.datetime.now().date())
    prev_trading_date = tw_calendar.previous_close(date).date()

    if date not in tw_calendar.opens:
        print('今日非交易日')
        return

    if is_Futures:
        code = ''.join(char for char in codes if char.isalpha())
    
    else:
        code = codes

    ticks[code] = get_ticks(connection, api, date, codes , is_Futures )[0]
    ticks[code] = ticks[code][['close','volume']]
    kbars[code] = get_MA(ticks_to_kbars(ticks[code], interval='30Min'))
    kbars[code] = kbars[code][prev_trading_date:]
    volume_today[code] = kbars[code]['volume'].sum()

    if is_Futures:
        api.quote.subscribe(api.Contracts.Futures.get(codes[0:3])[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
        

    else:
        api.quote.subscribe(api.Contracts.Stocks[codes],
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1
                            )
    
    while True:

        time.sleep(1)

        current_time = datetime.datetime.now()

        if (current_time.minute == 0 or current_time.minute == 30) and current_time.second == 0:

            kbars[code] = get_MA((ticks_to_kbars(ticks[code], interval='30Min')))
            kbars[code] = kbars[code][date:]

            str_current_time = (current_time - datetime.timedelta(minutes=30)).strftime('%Y-%m-%d %H:%M:%S')
                
            try:
                    
                current_price = kbars[code].loc[str_current_time]['close']
                
                if current_price >= EH:
                    EH = current_price
            except:
               continue
            
        
            open_order = open_orders.get(code, None)

            # Check the order status
            if open_order:

                api.update_status(api.futopt_account)

                if open_order.status.status in ['PendingSubmit', 'Submitted', 'Filling']:

                   api.cancel_order(open_order)

                   open_orders[code] = None
                   order_qty[code] = 0

                if open_order.status.status != 'Filling':
                    break

            position = positions.get(code, 0)

            if (
                #first condition
                position == 0 and 
                #OR condition
                #current_price/kbars.iloc[ts]['ma20']-1 < filter or 
                current_price > kbars.iloc[ts]['ma20']                        
            ):
                if (
                    #Entry condition
                    current_price > kbars.iloc[ts]['ma10']         
                ):
                    qty = 1

                    contract = api.Contracts.Futures.get(codes)[code]
                    order = api.Order(
                                    action=sj.constant.Action.Buy, 
                                    price=current_price,
                                    quantity=qty,
                                    price_type="LMT",
                                    order_type="ROD",
                                    octype=sj.constant.FuturesOCType.Auto,
                                    account=api.futopt_account)

                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = qty

                    entry_time = pd.Timestamp(date) + pd.Timedelta(hours=current_time.hour, minutes=current_time.minute, seconds=current_time.second)
                    entry_price = current_price

                    #Cost record
                    AC = current_price
                    EH = current_price
                    last_price = current_price
                    
                    #Report
                    print( 'Entry: [{}] buy {} at {}'.format(current_time, codes, current_price))

            elif(
                position != 0
            ):
                add_position = 1
                cost_after_add = (AC * position + current_price * add_position) / (position + add_position)
                cv1 = EH * (1+Ratio_Drawdown1) / cost_after_add -1
                cv2 = EH * (1 + Ratio_Drawdown1) 
                
                # 加碼條件1
                if cv1 > 0:
                    qty = add_position

                    contract = api.Contracts.Futures.get(codes)[code]
                    order = api.Order(
                                    action=sj.constant.Action.Buy, 
                                    price=current_price,
                                    quantity=qty,
                                    price_type="LMT",
                                    order_type="ROD",
                                    octype=sj.constant.FuturesOCType.Auto,
                                    account=api.futopt_account)

                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = qty

                    #Cost record
                    AC = (AC * (position-add_position) + current_price * add_position) / position
                    EH = current_price
                    last_price = current_price

                        
                    #Report
                    print( 'Entry: [{}] Reach condition1: {} and buy {} at {}'.format(current_time,  round(cv1, 5),
                                                                                    codes, current_price))
                    print( 'Now: [{}] buy {} for {} units in at average price of {} '.format(current_time, codes, position, round(AC,2)))
                
                #加碼條件2
                elif cv2 > last_price:
                    qty = add_position

                    contract = api.Contracts.Futures.get(codes)[code]
                    order = api.Order(
                                    action=sj.constant.Action.Buy, 
                                    price=current_price,
                                    quantity=qty,
                                    price_type="LMT",
                                    order_type="ROD",
                                    octype=sj.constant.FuturesOCType.Auto,
                                    account=api.futopt_account)

                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = qty

                    #Cost record
                    AC = (AC * (position-add_position) + current_price * add_position) / position
                    EH = current_price
                    last_price = current_price
                        
                    #Report
                    print( 'Entry: [{}] Reach condition2: {} and buy {} at {}'.format(current_time,  round(cv2, 5),
                                                                                    codes, current_price))
                    print( 'Now: [{}] buy {} for {} units in at average price of {} '.format(current_time, codes, position, round(AC,2)))
                    
                    #出場條件1
                elif(
                    position < 5 and
                    current_price <= EH * (1+Ratio_Drawdown2)
                ):
                    contract = api.Contracts.Futures.get(codes)[code]
                    order = api.Order(
                                    action=sj.constant.Action.Sell, 
                                    price=current_price,
                                    quantity=positions,
                                    price_type="LMT",
                                    order_type="ROD",
                                    octype=sj.constant.FuturesOCType.Auto,
                                    account=api.futopt_account
                                    )

                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = position

                    print('Exit: [{}] Reach condition1: {} and sell {} for {} units at {}'.format(current_time,
                                                                                                   round(EH * (1+Ratio_Drawdown1), 5),
                                                                                                  codes, position, current_price))
                    
                elif(
                    position >= 5 and
                    current_price <= EH * (1+Ratio_Drawdown2)
                ):
                    contract = api.Contracts.Futures.get(codes)[code]
                    order = api.Order(
                                    action=sj.constant.Action.Sell, 
                                    price=current_price,
                                    quantity=positions,
                                    price_type="LMT",
                                    order_type="ROD",
                                    octype=sj.constant.FuturesOCType.Auto,
                                    account=api.futopt_account
                                    )

                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = position

                    print( 'Exit: [{}] Reach condition2: {} and sell {} for {} units at {}'.format(current_time, 
                                                                                                  round(EH * (1+Ratio_Drawdown2),5),
                                                                                                  codes, position, current_price))
                break
if is_Futures:
    api.quote.subscribe(api.Contracts.Futures.get(codes[0:3])[codes], 
                        quote_type= 'tick',
                        version = sj.constant.QuoteVersion.v1 
                           )
        

else:
    api.quote.subscribe(api.Contracts.Stocks[codes],
                       quote_type= 'tick',
                       version = sj.constant.QuoteVersion.v1
                        )
    


with open('trade_historical_variable.pkl', 'wb') as file:
    trade_his_var = {
                    'Entry_time': entry_time,
                    'Entry_P':entry_price,
                    'last_P': last_price,
                    'AC': AC,
                    'EH': EH,
                    'cost_after_add':cost_after_add,
                    'Position': positions.get(code, 0)}

    pickle.dump(trade_his_var, file)

print("資料已保存")
    
print('----- 關閉 Tbot  #{} -----'.format(code))                                                                                 

NameError: name 'is_Futures' is not defined